In [1]:
import pandas as pd
import numpy as np

def _tp_score_effectiveness(col=""):
    def row_wise(row):
        gt_eff = row.discourse_effectiveness # ('Adequate' = 0, 'Effective' = 1, 'Ineffective' = 2)
        return row[f'score_discourse_effectiveness_{gt_eff}']
    return row_wise

def calc_overlap(set_pred, set_gt,threshold=0.5):
    """
    Calculates if the overlap between prediction and
    ground truth is enough fora potential True positive
    """
    # Length of each and intersection
    try:
        len_gt = len(set_gt)
        len_pred = len(set_pred)
        inter = len(set_gt & set_pred)
        overlap_1 = inter / len_gt
        overlap_2 = inter/ len_pred
        return overlap_1 >= threshold and overlap_2 >= threshold,max(overlap_1,overlap_2)
    except:  # at least one of the input is NaN
        return False,0

def score_feedback_comp_micro(pred_df, gt_df, discourse_type,threshold=0.5,weight_tp_segment=0.5):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df.loc[gt_df['discourse_type'] == discourse_type, 
                      ['id', 'predictionstring','discourse_effectiveness']].reset_index(drop=True)
    pred_df = pred_df.loc[pred_df['discourse_type'] == discourse_type,
                      ['id', 'predictionstring','score_discourse_effectiveness_0',
                                               'score_discourse_effectiveness_1',
                                               'score_discourse_effectiveness_2']].reset_index(drop=True)
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    pred_df['predictionstring'] = [set(pred.split(' ')) for pred in pred_df['predictionstring']]
    gt_df['predictionstring'] = [set(pred.split(' ')) for pred in gt_df['predictionstring']]
    
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on='id',
                           right_on='id',
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    
    overlaps = [calc_overlap(*args,threshold=threshold) for args in zip(joined.predictionstring_pred, 
                                                     joined.predictionstring_gt)]
    
    joined['overlaps'] = np.asarray([x[0] for x in overlaps])*1
    joined['overlaps_scores'] = np.asarray([x[1] for x in overlaps])*1
    joined['effectiveness_TP_score'] = joined.apply(_tp_score_effectiveness(' '),axis=1)
    joined['1_effectiveness_TP_score'] = 1-joined['effectiveness_TP_score']
    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    # we don't need to compute the match to compute the score
    
    joined = joined.sort_values(["overlaps",'overlaps_scores'],ascending=False).reset_index(drop=True).groupby('gt_id').head(1)
    
    TP = joined[joined.overlaps==1]['gt_id'].nunique()
    
    TP_weighted = weight_tp_segment*TP + (1-weight_tp_segment)*(joined[joined.overlaps==1]['effectiveness_TP_score'].sum())
    
    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    TPandFP = len(pred_df)
    TPandFN = len(gt_df)
    
    TPandFP_weighted = TP_weighted + (joined[joined.overlaps==0]['1_effectiveness_TP_score'].sum())
    
    #calc microf1
    f1_score_fb1 = 2*TP / (TPandFP + TPandFN)
    new_f1_score = 2*TP_weighted / (TPandFP_weighted + TPandFN)
    return f1_score_fb1,new_f1_score

def score_feedback_comp(pred_df, gt_df,threshold=0.5, weight_tp_segment=0.5,return_class_scores=False):
    class_scores_fb1 = {}
    new_class_scores = {}
    for discourse_type in gt_df.discourse_type.unique():
        s_fb1,s = score_feedback_comp_micro(pred_df, gt_df, discourse_type,threshold,weight_tp_segment)
        class_scores_fb1[discourse_type] = s_fb1
        new_class_scores[discourse_type] = s
    
    f1_fb1 = np.mean([v for v in class_scores_fb1.values()])
    new_f1 = np.mean([v for v in new_class_scores.values()])
    if return_class_scores:
        return f1_fb1,class_scores_fb1,new_f1, new_class_scores
    return f1_fb1,new_f1

In [2]:
gt = pd.DataFrame({'id':'963859B3739B',
                   "discourse_type":[1, 2, 2, 2, 2, 5, 2, 5, 6],
                   "discourse_effectiveness":[2,2,0,0,0,0,0,0,0],
                   "predictionstring":['0 1 2 3 4 5 6 7 8',
                                       '16 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
                                       '32 33 34 35 36 37 38 39 40 41', '46 47 48 49 50 51',
                                       '63 64 65 66 67 68 69 70 71 72 73 74 75',
                                       '76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92',
                                       '93 94 95 96 97 98 99 100 101',
                                       '158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390',
                                       '391 392 393 394 395 396 397 398 399']
                  })
gt

,id,discourse_type,discourse_effectiveness,predictionstring
0,963859B3739B,1,2,0 1 2 3 4 5 6 7 8
1,963859B3739B,2,2,16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
2,963859B3739B,2,0,32 33 34 35 36 37 38 39 40 41
3,963859B3739B,2,0,46 47 48 49 50 51
4,963859B3739B,2,0,63 64 65 66 67 68 69 70 71 72 73 74 75
5,963859B3739B,5,0,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
6,963859B3739B,2,0,93 94 95 96 97 98 99 100 101
7,963859B3739B,5,0,158 159 160 161 162 163 164 165 166 167 168 16...
8,963859B3739B,6,0,391 392 393 394 395 396 397 398 399


In [3]:
pred_df = gt.copy()
pred_df[['score_discourse_effectiveness_0',
         'score_discourse_effectiveness_1',
         'score_discourse_effectiveness_2']] = 1

In [4]:
pred_df

,id,discourse_type,discourse_effectiveness,predictionstring,score_discourse_effectiveness_0,score_discourse_effectiveness_1,score_discourse_effectiveness_2
0,963859B3739B,1,2,0 1 2 3 4 5 6 7 8,1,1,1
1,963859B3739B,2,2,16 17 18 19 20 21 22 23 24 25 26 27 28 29 30,1,1,1
2,963859B3739B,2,0,32 33 34 35 36 37 38 39 40 41,1,1,1
3,963859B3739B,2,0,46 47 48 49 50 51,1,1,1
4,963859B3739B,2,0,63 64 65 66 67 68 69 70 71 72 73 74 75,1,1,1
5,963859B3739B,5,0,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,1,1,1
6,963859B3739B,2,0,93 94 95 96 97 98 99 100 101,1,1,1
7,963859B3739B,5,0,158 159 160 161 162 163 164 165 166 167 168 16...,1,1,1
8,963859B3739B,6,0,391 392 393 394 395 396 397 398 399,1,1,1


In [5]:
f1_fb1 , new_f1 = score_feedback_comp(pred_df, 
                                      gt,
                                      threshold=0.5,
                                      weight_tp_segment=0.5, return_class_scores=False)

In [6]:
f1_fb1 , new_f1

(1.0, 1.0)

In [ ]:
if not os.path.exists("../input/train_cache.csv"):
        df = pd.read_csv(f"{cfg.dataset.base_dir}/persuade_corpus.csv")
        df['id'] = df['essay_id']
        df_train = df.loc[df['competition_set'] == "train"].reset_index(drop=True)
        df_train = df_train.loc[df_train['discourse_type'] != "Unannotated"].reset_index(drop=True)
        df_test = df.loc[df['competition_set'] == "test"].reset_index(drop=True)
        df_test = df_test.loc[df_test['discourse_type'] != "Unannotated"]
        df_test = df_test[~df_test.discourse_effectiveness.isna()].reset_index(drop=True)
        df_test['id'] = df_test['essay_id'] 
        df_test['Ineffective'] = df_test.discourse_effectiveness.apply(lambda x: 1 if x == "Ineffective" else 0)
        df_test['Adequate'] = df_test.discourse_effectiveness.apply(lambda x: 1 if x == "Adequate" else 0)
        df_test['Effective'] = df_test.discourse_effectiveness.apply(lambda x: 1 if x == "Effective" else 0)

        
        df_test_pri = df_test.loc[df_test['test_split_feedback_1'] == "Private"].reset_index(drop=True)
        df_test = df_test.loc[df_test['test_split_feedback_1'] == "Public"].reset_index(drop=True)


        fb1 = df_train.loc[df_train['in_feedback2.0'] == 0].reset_index(drop=True)
        fb1_effective = fb1.loc[fb1['discourse_effectiveness'] == "Effective"].reset_index(drop=True)
        fb1_adequate = fb1.loc[fb1['discourse_effectiveness'] == "Adequate"].reset_index(drop=True)
        fb2 = df_train.loc[df_train['in_feedback2.0'] == 1].reset_index(drop=True)

        df = pd.concat([fb1_adequate, fb1_effective, fb2]).reset_index(drop=True)

        data = []
        for essay_id in tqdm(df.essay_id.unique(), total=len(df.essay_id.unique())):
            temp_df = df[df.essay_id == essay_id].reset_index(drop=True)
            res = utils.add_discourse_start_end(temp_df, datatype="train")
            data.append(res)

        df_train = pd.concat(data).reset_index(drop=True)

        data_test = []
        for essay_id in tqdm(df_test.essay_id.unique(), total=len(df_test.essay_id.unique())):
            temp_df = df_test[df_test.essay_id == essay_id].reset_index(drop=True)
            res = utils.add_discourse_start_end(temp_df, datatype="train")
            data_test.append(res)

        df_test = pd.concat(data_test).reset_index(drop=True)  
        df_train = df_train[df_train.discourse_start != -1].reset_index(drop=True)
        df_test = df_test[df_test.discourse_start != -1].reset_index(drop=True)
        
        df_train.to_csv("../input/train_cache.csv", index=False)
        df_test.to_csv("../input/test_cache.csv", index=False)
    else:
        df_train = pd.read_csv("../input/train_cache.csv")
        df_test = pd.read_csv("../input/test_cache.csv")